In [1]:
import matplotlib, os
import numpy as np
import imageio.v3 as imageio
from IPython.display import clear_output
from shared.Render import collect_saved_frames, render_function_point
from shared.TestFunctions import TEST_FUNCTIONS
matplotlib.use('Agg')
np.random.seed(42)

#### The differential evolution algorithm

In [2]:
def differential_evolution(func, rnge: tuple, num_individuals = 30, iterations = 30, f = 0.8, cr = 0.9):
    population = [
        (np.random.uniform(rnge[0], rnge[1]), np.random.uniform(rnge[0], rnge[1]))
        for _ in range(num_individuals)
    ]
    values = [func((x, y)) for x, y in population]
    for idx, point in enumerate(population): 
        yield 0, idx, point
    for iter in range(1, iterations + 1):
        new_population = population.copy()
        new_values = values.copy()
        for i in range(num_individuals):
            indices = list(range(num_individuals))
            indices.remove(i)
            r1, r2, r3 = np.random.choice(indices, 3, replace=False)
            mutation_vec = np.clip(np.array([
                (population[r1][0] - population[r2][0]) * f + population[r3][0],
                (population[r1][1] - population[r2][1]) * f + population[r3][1],
            ]), rnge[0], rnge[1])
            trial_vec = np.zeros(2)
            j_rnd = np.random.randint(0, 2)
            for j in range(2):
                if np.random.uniform() < cr or j == j_rnd:
                    trial_vec[j] = mutation_vec[j]
                else:
                    trial_vec[j] = population[i][j]
            trial_z = func(trial_vec)
            if trial_z <= values[i]:
                new_population[i] = trial_vec
                new_values[i] = trial_z
                yield iter, i, trial_vec
        population = new_population
        new_values = new_values

#### Run the differential evolution algorithm and collect frames for multiple functions

In [3]:
gif_folder_path = "gifs/differential_evolution"
os.makedirs(gif_folder_path, exist_ok=True)

In [4]:
for func, params in TEST_FUNCTIONS.items():
    print(f"Test function: {func.__name__}")
    rnge = (params[0], params[1])
    resolution = params[2]
    for iter, idx, point in differential_evolution(func, rnge):
        frame = render_function_point(func, rnge=rnge, resolution=resolution, point=point, iteration=f"{iter} - {idx}")
        imageio.imwrite(os.path.join(gif_folder_path, f"{func.__name__}{iter:08d}{idx:08d}.png"), frame)
    collect_saved_frames(gif_folder_path, f"{str.capitalize(func.__name__)}.gif", fps=5)
clear_output()

#### Visualize the differential evolution algorithm on multiple functions

![Sphere](gifs/differential_evolution/Sphere.gif)
![Ackley](gifs/differential_evolution/Ackley.gif)
![Rastrigin](gifs/differential_evolution/Rastrigin.gif)
![Rosenbrock](gifs/differential_evolution/Rosenbrock.gif)
![Griewank](gifs/differential_evolution/Griewank.gif)
![Schwefel](gifs/differential_evolution/Schwefel.gif)
![Levy](gifs/differential_evolution/Levy.gif)
![Michalewicz](gifs/differential_evolution/Michalewicz.gif)
![Zakharov](gifs/differential_evolution/Zakharov.gif)